In [ ]:
#Import and store the data in a data frame

import pandas as pd
air = pd.read_csv('airline.csv')
print(air.shape)

In [ ]:
#Remove the outliers from the dataRemove the outliers from the data

import numpy as np
import pandas as pd
Q1 = air.quantile(0.25)
Q3 = air.quantile(0.75)
IQR = Q3-Q1
air= air[~((air< (Q1 - 1.5 * IQR)) | (air> (Q3 + 1.5 * IQR))).any(axis=1)]
print(air.shape)

In [ ]:
#Treat “Average Fare” – 3rdColumn as your Dependent Variable and Rest of the columns as Independent Variable.

x=air.loc[:, air.columns != 'Average Fare']
y=air.loc[:, air.columns == 'Average Fare']
x= x.fillna(0)
y=y.fillna(0)

In [ ]:
#Drop the independent variables which has less than 0.1 correlation with the dependent variable 

to_drop= air.corr() ['Average Fare'] .abs()
to_drop = to_drop.fillna(0)
for i in range(to_drop.shape[0]):
    if to_drop[i] < 0.1:
        col=to_drop.index[i]
        x.drop([col], axis='columns',inplace=True)

In [ ]:
#Create scatter Plot of Independent Variable vs Dependent Variable.

%matplotlib inline
import matplotlib.pyplot as plt
#import matplotlib
for i in range(len(x.columns)):
    if x[x.columns[i]].dtype in ('int64','float64'):
        air.plot.scatter(x=x.columns[i],y='Average Fare')

In [ ]:
#Divide the data set into training and test data set and build a Multiple Linear Regression model

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)
regressor = LinearRegression()
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

results = pd.DataFrame({'Actual': [y_test], 'Predicted': [y_pred]})
print(results)

In [ ]:
#Print the coefficients & intercepts of the linear regression model 

print("Coefficient: ",regressor.coef_)
print("Intercept: ",regressor.intercept_)

In [ ]:
#Print the accuracy of the overall model

from sklearn.metrics import r2_score
print('Accuracy Score: ', (r2_score(y_test, y_pred)))